#### Functions (IGNORE)

In [2]:
import yfinance as yf
missing_data_tickers = [] # use this as a list of tickers with missing data

def get_data_from_start_to_end(ticker, start_date, end_date):
    global missing_data_tickers  # Use the global list to accumulate missing tickers
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if stock_data.empty:
            missing_data_tickers.append(ticker)
            raise ValueError(f"Stock data for ticker {ticker} during the period from {start_date} to {end_date} was not found.")
        return stock_data
    except Exception as e:
        print(f"An error occurred for ticker {ticker}: {e}")
        missing_data_tickers.append(ticker)
        return None


In [3]:
# for a variety of periods load in different list of tickers
def download_stock_data_for_periods(tickers, periods):
    all_data = {}
    
    for period, (start_date, end_date) in periods.items():
        period_data = {}
        for ticker in tickers:
            data = get_data_from_start_to_end(ticker, start_date, end_date)
            if data is not None:
                period_data[ticker] = data
        all_data[period] = period_data
    
    return all_data

In [4]:
import pandas as pd

# Get the adjusted close prices
adj_close_sector_etf = {}

# Create adjusted close price only listing of sector ETFs
def get_adjusted_closed_price(nested_dict, tickers, periods):
    for period in periods:
        stock_price_df = pd.DataFrame()  # Create a new DataFrame for each period
        for ticker in tickers:
            stock_price_df[ticker] = nested_dict[period][ticker]['Adj Close']
        
        adj_close_sector_etf[period] = stock_price_df  # Store the complete DataFrame for the period
    
    return adj_close_sector_etf

In [5]:
import random

def stochastic_modeling(nested_dict, tickers, periods,num_samples):
    # Store the returns in a nested dictionary
    nested_dict_returns = {period: {ticker: [] for ticker in tickers} for period in periods}

    # Go through each economic time period
    for period in periods:
        max_index = len(nested_dict[period]) - 90  # Ensure there's enough data to calculate ROI

        # Generate random samples from the valid range
        random_dates = random.choices(range(max_index), k=num_samples)

        for ticker in tickers:
            for date_idx in random_dates:
                start_price = nested_dict[period][ticker].iloc[date_idx]
                end_price = nested_dict[period][ticker].iloc[date_idx + 90]

                # Get the return by the Holding Period Return
                roi = (((end_price - start_price) / start_price) * 100)

                nested_dict_returns[period][ticker].append(roi)

    return nested_dict_returns  # Return the nested dictionary with returns

In [6]:
def stochastic_roi(tickers,periods,return_rates_list,analysis_type):
    df = pd.DataFrame(index=tickers,columns=periods)
    for period in periods:
        for ticker in tickers:
            data = pd.Series(return_rates_list[period][ticker])
            if analysis_type=='Mean':
                df.at[ticker,period] = data.mean()
            elif analysis_type=='Median':
                df.at[ticker,period] = data.median()
            elif analysis_type=='Std':
                df.at[ticker,period] = data.std()
            elif analysis_type=='Variance':
                df.at[ticker,period] = data.var()

    return df

# Technical Analysis Investment Strategy

In [7]:
# import packages
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# create time periods for where this takes place
economic_cycle_periods = {

    "trough": ("2008-10-01", "2009-06-01"),
    "expansion": ("2012-01-01", "2015-01-01"),
    "peak": ("2019-06-01", "2020-02-01"),
    "contraction": ("2007-12-01", "2008-10-01"),
}

economic_cycle_periods_list = ['trough','expansion','peak','contraction']

In [9]:
# create etf tickers for sectors
sector_etf_tickers = [
    'XLB', # materials sector
    'XLI', # industrials sector
    'XLF', # financials
    'XLK', # information technology
    'XLY', # consumer discretionary
    'XLP', # consumer staples
    'XLE', # energy
    'XLV', # healthcare
    'VOX', # communication services
    'XLU', # utilities
    'IYR' # real estate
    ]

## Buy and Hold Investment Technique
The buy and hold strategy is a passive investing strategy that will be applied to the 11 sector ETFs during different macroeconomic time periods.

In [10]:
# save nested dictionary data as a variable to be accessed.
sector_etf_data = download_stock_data_for_periods(sector_etf_tickers,economic_cycle_periods)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [11]:
# get adjusted close price
sector_etf_adjusted_close = get_adjusted_closed_price(sector_etf_data,sector_etf_tickers,economic_cycle_periods_list)

### Perform stochastic modeling using buy and hold strategy
Use a different day where the stock begins investing then hold for a month and see the return.

In [12]:
# perform stochastic modeling on the buy and 
stochastic_buy_hold = stochastic_modeling(sector_etf_adjusted_close,sector_etf_tickers,economic_cycle_periods_list,10000)

In [13]:
# this can be repeated for mean, median, std and var
stochastic_roi(sector_etf_tickers,economic_cycle_periods_list,stochastic_buy_hold,'Mean')

,trough,expansion,peak,contraction
XLB,-1.318429,4.947896,4.122003,-0.410008
XLI,-13.395348,6.454556,6.838566,-4.574048
XLF,-22.136185,7.326857,9.574088,-15.538975
XLK,3.250878,5.527304,12.575215,-2.801214
XLY,-1.336732,7.100409,3.415217,-4.212039
XLP,-8.296753,5.797062,5.170427,0.524815
XLE,-4.974294,3.581739,-0.394792,3.405652
XLV,-4.705847,9.150425,8.456494,-3.744825
VOX,4.057455,5.475113,6.113578,-5.425362
XLU,-8.707737,4.855298,6.29711,-2.709144


## Bollinger Bands Investment Technique
Using John Bollinger's techniques 'Bollinger Bands' to create buy and sell signals to observe the roi for investing for a month.

In [14]:
# add bollinger data
import scipy.stats as stats
def add_bollinger_data(data,window,conf_int):
        z_score = stats.norm.ppf(1 - (1 - conf_int) / 2) # create a zscore from the mean

        data['middle_band'] = data['Adj Close'].rolling(window).mean()
        data['upper_band'] = data['middle_band'] + z_score * data['Adj Close'].rolling(window).std()
        data['lower_band'] = data['middle_band'] - z_score * data['Adj Close'].rolling(window).std()

        data['Signal'] = None

        data['Signal'] = np.where(data['Adj Close'] < data['lower_band'], 'Buy', 
                              np.where(data['Adj Close'] > data['upper_band'], 'Sell', np.nan))

        return data

In [15]:
# create bollinger data for multiple time period and multiple tickers
def bollinger_data_multiple_periods_tickers(periods,tickers,data,window,confidence_period):
    # for each ticker in economic time periods
    for period in periods:
            for ticker in tickers:
                    try:
                        add_bollinger_data(data[period][ticker],window,confidence_period)
                    except KeyError:
                        print(f'Data for {ticker} does not exist during {period}')

In [16]:
# create bollinger bands in stock data
bollinger_data_multiple_periods_tickers(economic_cycle_periods_list,sector_etf_tickers,sector_etf_data,20,0.95)
sector_etf_data['trough']['XLB']

,Open,High,Low,Close,Adj Close,Volume,middle_band,upper_band,lower_band,Signal
Date,,,,,,,,,,
2008-10-01,32.759998,33.189999,32.130001,32.849998,23.119270,14639500,NaN,NaN,NaN,nan
2008-10-02,31.540001,31.860001,29.930000,30.490000,21.458342,12581300,NaN,NaN,NaN,nan
2008-10-03,30.190001,31.690001,29.780001,30.190001,21.247200,16770600,NaN,NaN,NaN,nan
2008-10-06,29.510000,29.510000,26.889999,28.700001,20.198568,22512700,NaN,NaN,NaN,nan
2008-10-07,29.160000,29.530001,27.049999,27.219999,19.156971,16004900,NaN,NaN,NaN,nan
...,...,...,...,...,...,...,...,...,...,...
2009-05-22,26.530001,26.660000,26.110001,26.299999,18.848707,8421500,18.742287,19.907891,17.576683,nan
2009-05-26,26.170000,26.969999,25.830000,26.930000,19.300222,7886900,18.827572,19.888616,17.766528,nan
2009-05-27,26.790001,26.850000,25.860001,25.920000,18.576380,7216600,18.881323,19.759550,18.003096,nan


In [17]:
# example case of bollinger bands in stock data
sector_etf_data['expansion']['XLB']['Signal'].value_counts()

nan     670
Buy      45
Sell     39
Name: Signal, dtype: int64

In [18]:
def collect_signals(nested_dict, periods, tickers):
    # Initialize an empty dictionary to hold DataFrames for each period
    bb_nested_dict = {}

    for period in periods:
        # Create a DataFrame for each period with the tickers as columns
        signals_period = pd.DataFrame(columns=tickers)
        
        # Loop through each ticker and extract the 'Signal'
        for ticker in tickers:
            signals_period[ticker] = nested_dict[period][ticker]['Signal']
        
        # Store the DataFrame in the dictionary using the period as the key
        bb_nested_dict[period] = signals_period

    # Return the dictionary containing DataFrames for each period
    return bb_nested_dict

In [19]:
bb_bands_signals = collect_signals(sector_etf_data,economic_cycle_periods_list,sector_etf_tickers)
bb_bands_signals['trough']

,XLB,XLI,XLF,XLK,XLY,XLP,XLE,XLV,VOX,XLU,IYR
Date,,,,,,,,,,,
2008-10-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2008-10-02,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2008-10-03,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2008-10-06,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2008-10-07,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...
2009-05-22,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2009-05-26,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2009-05-27,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [20]:
# make sure that the length of the two dataframes are same
# this is so that you can treat them as two panes stacked on top of one another and each index is relevant to the date and ticker
len(sector_etf_adjusted_close) == len(bb_bands_signals)

True

In [21]:
from datetime import timedelta

# goal is to create a function that uses the bb signal df and adj close signal df
# you run through the signal day by day ticker by ticker
# get the buy/sell/hold action
# dependent on action get the adj close price
# invest/sell a certain amount based on how much cash is available
# perform stocastic modeling similar to buy and hold to see how the investment changes for each different sectors in different time periods

# create function for signals 
def bb_band_roi(bb_signals_nd, adj_close_nd,periods_date,periods_list,tickers,n_sample,initial_investment,percent_to_buy,percent_to_sell):
    
    # track the investment
    invesment_tracker = pd.DataFrame(columns=[['date','investment value','account balance','profit']])

    # track the actions made on a given day
    stock_actions = pd.DataFrame(columns=[['action_stocks','buy_sell_amount ($)','buy_sell_amount(number)',]])

    # track the stocks invested each day
    # needs to be a nested dictionary where every every stock has a day by day tracker
    portfolio_summary = {period: {ticker: pd.DataFrame() for ticker in tickers} for period in periods_list}
    
    # create a nested dictionary with technical analysis signals and adj close price as pages
    all_data = {
        'Adj Close': adj_close_nd,
        'Bollinger Band': bb_signals_nd
    }

     # set the data that is going to be accessed
    adj_close_data = all_data['Adj Close']
    bollinger_band_data = all_data['Bollinger Band']

    # go through each period
    for period in periods_list:
            
            # create a tracker for just the investment
        

            # Initialize current balance and investment tracking
            portfolio_balance = initial_investment
            shares_value = 0
            shares_number = 0
            investment_value = initial_investment
            profit = 0
            
            # Track investments
            investment_tracker = []

            

            # create the date range to be sampled from
            date_range = pd.date_range(start=pd.to_datetime(periods_date[period][0]),end=pd.to_datetime((periods_date[period][1]))-timedelta(days=90))

            # get the random dates for stochastic modeling
            start_dates = np.random.choice(date_range, size=n_sample, replace=False)

            for time_stamp in start_dates:
                time_stamp = pd.to_datetime(time_stamp)

                amount_to_buy = percent_to_buy * portfolio_balance
                
                # get the data based on the start and end dates
                adj_close_period = adj_close_data[period].loc[time_stamp:time_stamp + timedelta(days=90)]
                bb_signals_period = bollinger_band_data[period].loc[time_stamp:time_stamp + timedelta(days=90)]

                # go through each day in the signals then collect the location
                for row_idx, row in bb_signals_period.iterrows():
                    for col_idx,value in enumerate(row):
                        adj_close_price = adj_close_period.loc[row_idx, tickers[col_idx]]
                        if value=='Buy':
                            # get the amount to buy
                            amount_to_buy = percent_to_buy * portfolio_balance

                            # keep track of investment as a whole
                            

                            # if the balance in the portfolio is greater than the amount to buy
                        if portfolio_balance >= amount_to_buy:
                            amount_to_buy = percent_to_buy * portfolio_balance
                            shares_to_buy = amount_to_buy / adj_close_price
                            shares_number += shares_to_buy
                            shares_value = shares_number * adj_close_price
                            portfolio_balance -= amount_to_buy
                            investment_value = shares_value + portfolio_balance
                            profit = investment_value - initial_investment
                            
                            
                            investment_tracker.append({
                                'Date': col_idx,
                                'Share Price': adj_close_price,
                                'Signal': 'Buy',
                                'Buy/Sell Amount ($)': amount_to_buy,
                                'Buy/Sell Number of Shares': shares_to_buy,
                                'Shares ($) Ownership': shares_value,
                                'Shares Ownership': shares_number,
                                'Account Balance': portfolio_balance,
                                'Investment Value': investment_value,
                                'Profit': profit
                            })

                        elif value == 'Sell':
                            amount_to_sell = percent_to_sell * portfolio_balance # made a change to make it portfolio balance rather than amount in stock account
                            # Check if there are enough shares to sell
                            if shares_value >= amount_to_sell:
                                shares_to_sell = amount_to_sell / adj_close_price
                                shares_number -= shares_to_sell
                                shares_value = shares_number * adj_close_price
                                portfolio_balance += amount_to_sell
                                investment_value = shares_value + portfolio_balance
                                profit = investment_value - initial_investment

                                investment_tracker.append({
                                    'Date': col_idx,
                                    'Share Price': adj_close_price,
                                    'Signal': 'Sell',
                                    'Buy/Sell Amount ($)': amount_to_sell,
                                    'Buy/Sell Number of Shares': shares_to_sell,
                                    'Shares ($) Ownership': shares_value,
                                    'Shares Ownership': shares_number,
                                    'Account Balance': portfolio_balance,
                                    'Investment Value': investment_value,
                                    'Profit': profit
                            })
                            
                        else: 
                            investment_value = (shares_number * adj_close_price) + portfolio_balance
                            shares_value = shares_number * adj_close_price
                            profit = investment_value - initial_investment

                            investment_tracker.append({
                                'Date': col_idx,
                                'Share Price': adj_close_price,
                                'Signal': np.nan,
                                'Buy/Sell Amount ($)': 0,
                                'Buy/Sell Number of Shares': 0,
                                'Shares ($) Ownership': shares_value, 
                                'Shares Ownership': shares_number, 
                                'Account Balance': portfolio_balance, 
                                'Investment Value': investment_value,
                                'Profit': profit
                        })

                    # Store the investment history in the dictionary
                    portfolio_summary[period][tickers[col_idx]] = pd.DataFrame(investment_tracker)

                    all_data.update(portfolio_summary)

                    return portfolio_summary

a = bb_band_roi(bb_bands_signals,adj_close_sector_etf,economic_cycle_periods,economic_cycle_periods_list,sector_etf_tickers,1,1000,0.05,0.01)

    # next step is to store the investment data and keep track of the portofolio
            

In [27]:
portfolio_summary = {period: {ticker: pd.DataFrame() for ticker in sector_etf_tickers} for period in economic_cycle_periods_list}
portfolio_summary['trough']['XLB']

""


In [50]:
import pandas as pd
import numpy as np
from datetime import timedelta

def bb_band_roi(bb_signals_nd, adj_close_nd, periods_date, periods_list, tickers, n_sample, initial_investment, percent_to_buy, percent_to_sell):

    # Track actions day by day
    portfolio_tracker = {period: None for period in periods_list}

    # Portfolio summary - nested dictionary for each period and ticker
    portfolio_summary = {period: {ticker: pd.DataFrame() for ticker in tickers} for period in periods_list}
    
    # Set data to be accessed
    adj_close_data = adj_close_nd
    bollinger_band_data = bb_signals_nd

    all_data = {
        'Stock Tracker': portfolio_summary,
        'Portfolio Tracker': portfolio_tracker,
        'Adjusted Close Price': adj_close_nd,
        'Bollinger Band Signal':bollinger_band_data
    }

    # Loop through each economic period
    for period in periods_list:

        # Create the date range for the current period
        date_range = pd.date_range(start=pd.to_datetime(periods_date[period][0]), end=pd.to_datetime(periods_date[period][1]) - timedelta(days=90))
        
        # Get random dates for stochastic modeling
        start_dates = np.random.choice(date_range, size=n_sample, replace=False)

        # Loop through sampled start dates
        for start_date in start_dates:
            time_stamp = pd.to_datetime(start_date)

            # Initialize balance for portfolio investment   
            account_balance = initial_investment
            portfolio_value = 0
            profit = 0
            shares_number = 0
            shares_value = 0

            # Extract the adjusted close and signal data for time period
            adj_close_period = adj_close_data[period].loc[time_stamp:time_stamp + timedelta(days=90)]
            bb_signals_period = bollinger_band_data[period].loc[time_stamp:time_stamp + timedelta(days=90)]

            # Iterate over each row in the Bollinger Band signals (day by day)
            for row_idx, row in bb_signals_period.iterrows():
                for col_idx, signal in enumerate(row):
                    adj_close_price = adj_close_period.loc[row_idx, tickers[col_idx]]  # Get corresponding adjusted close price

                    # initialize the tracking of stocks
                    stock_tracker = all_data['Stock Tracker'][period][sector_etf_tickers[col_idx]]
                
                # Handle Buy action
                if signal == 'Buy':
                    amount_to_buy = percent_to_buy * account_balance
                    if account_balance >= amount_to_buy:

                        # get the data for portfolio summary
                        shares_to_buy = amount_to_buy / adj_close_price
                        shares_number += shares_to_buy
                        shares_value = shares_number * adj_close_price

                        
                        #account_balance -= amount_to_buy
                        #portfolio_value = shares_value + account_balance
                        #profit = initial_investment - portfolio_value

                        # Track investment for the current period
                        stock_tracker = stock_tracker.append({
                            'Date': row_idx,
                            'Share Price': adj_close_price,
                            'Signal': 'Buy',
                            'Buy/Sell Amount ($)': amount_to_buy,
                            'Buy/Sell Number of Shares': shares_to_buy,
                            'Shares ($) Ownership': shares_value,
                            'Shares Ownership': shares_number,
                            #'Account Balance': portfolio_balance,
                            #'Investment Value': investment_value,
                            #'Profit': profit
                        }, ignore_index=True)

                # Handle Sell action
                elif signal == 'Sell':
                    amount_to_sell = percent_to_sell * account_balance
                    if shares_value >= amount_to_sell:

                        # get the data for portfolio summary
                        shares_to_sell = amount_to_sell / adj_close_price
                        shares_number -= shares_to_sell
                        shares_value = shares_number * adj_close_price


                        #account_balance += amount_to_sell
                        #investment_value = shares_value + account_balance
                        #profit = investment_value - initial_investment

                        # Track the sell action
                        stock_tracker = stock_tracker.append({
                            'Date': row_idx,
                            'Share Price': adj_close_price,
                            'Signal': 'Sell',
                            'Buy/Sell Amount ($)': amount_to_sell,
                            'Buy/Sell Number of Shares': shares_to_sell,
                            'Shares ($) Ownership': shares_value,
                            'Shares Ownership': shares_number,
                            #'Account Balance': portfolio_balance,
                            #'Investment Value': investment_value,
                            #'Profit': profit
                        }, ignore_index=True)

                # Handle Hold action (no action taken)
                else:
                    #investment_value = (shares_number * adj_close_price) + account_balance
                    #shares_value = shares_number * adj_close_price
                    #profit = investment_value - initial_investment

                    # Track the hold state
                    stock_tracker = stock_tracker.append({
                        'Date': row_idx,
                        'Share Price': adj_close_price,
                        'Signal': 'Hold',
                        'Buy/Sell Amount ($)': 0,
                        'Buy/Sell Number of Shares': 0,
                        'Shares ($) Ownership': shares_value,
                        'Shares Ownership': shares_number,
                        #'Account Balance': portfolio_balance,
                        #'Investment Value': investment_value,
                        #'Profit': profit
                    }, ignore_index=True)
            
            # Save the ticker's investment history into the portfolio summary for the current period
            all_data['Stock Tracker'][period][sector_etf_tickers[col_idx]] = stock_tracker.copy()

# Return the complete portfolio summary for all periods and tickers
    return all_data

# Example function call
a = bb_band_roi(bb_bands_signals, adj_close_sector_etf, economic_cycle_periods, economic_cycle_periods_list, sector_etf_tickers, 1, 1000, 0.05, 0.01)


/var/folders/6q/lw13gkln44z1r6ncfrmcbc5w0000gn/T/ipykernel_4117/3749965799.py:121: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_tracker = stock_tracker.append({
/var/folders/6q/lw13gkln44z1r6ncfrmcbc5w0000gn/T/ipykernel_4117/3749965799.py:121: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_tracker = stock_tracker.append({
/var/folders/6q/lw13gkln44z1r6ncfrmcbc5w0000gn/T/ipykernel_4117/3749965799.py:121: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_tracker = stock_tracker.append({
/var/folders/6q/lw13gkln44z1r6ncfrmcbc5w0000gn/T/ipykernel_4117/3749965799.py:121: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_track

In [53]:
a['Stock Tracker']['trough']

{'XLB': Empty DataFrame
 Columns: []
 Index: [],
 'XLI': Empty DataFrame
 Columns: []
 Index: [],
 'XLF': Empty DataFrame
 Columns: []
 Index: [],
 'XLK': Empty DataFrame
 Columns: []
 Index: [],
 'XLY': Empty DataFrame
 Columns: []
 Index: [],
 'XLP': Empty DataFrame
 Columns: []
 Index: [],
 'XLE': Empty DataFrame
 Columns: []
 Index: [],
 'XLV': Empty DataFrame
 Columns: []
 Index: [],
 'VOX': Empty DataFrame
 Columns: []
 Index: [],
 'XLU': Empty DataFrame
 Columns: []
 Index: [],
 'IYR':         Date  Share Price Signal  Buy/Sell Amount ($)  \
 0 2009-02-20    15.748575   Hold                    0   
 
    Buy/Sell Number of Shares  Shares ($) Ownership  Shares Ownership  
 0                          0            315.289092         21.305502  }

In [77]:
import pandas as pd
import numpy as np
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


def bb_band_roi(bb_signals_nd, adj_close_nd, periods_date, periods_list, tickers, n_sample, initial_investment, percent_to_buy, percent_to_sell):

    # Track actions day by day
    portfolio_tracker = {period: pd.DataFrame(columns=['Date', 'Account Balance', 'Portfolio Value', 'Total Value', 'Profit']) for period in periods_list}

    # Portfolio summary - nested dictionary for each period and ticker
    portfolio_summary = {period: {ticker: pd.DataFrame() for ticker in tickers} for period in periods_list}
    
    # Set data to be accessed
    adj_close_data = adj_close_nd
    bollinger_band_data = bb_signals_nd

    all_data = {
        'Stock Tracker': portfolio_summary,
        'Portfolio Tracker': portfolio_tracker,
        'Adjusted Close Price': adj_close_nd,
        'Bollinger Band Signal':bollinger_band_data
    }

    # Loop through each economic period
    for period in periods_list:

        # Create the date range for the current period
        date_range = pd.date_range(start=pd.to_datetime(periods_date[period][0]), end=pd.to_datetime(periods_date[period][1]) - timedelta(days=150))
        
        # Get random dates for stochastic modeling
        start_dates = np.random.choice(date_range, size=n_sample, replace=False)

        # Loop through sampled start dates
        for start_date in start_dates:
            time_stamp = pd.to_datetime(start_date)

            # Initialize balance for portfolio investment   
            account_balance = initial_investment
            portfolio_value = 0
            shares_number = {ticker: 0 for ticker in tickers}  # Initialize share count for each ticker
            shares_value = {ticker: 0 for ticker in tickers}   # Initialize share value for each ticker

            # Extract the adjusted close and signal data for time period
            adj_close_period = adj_close_data[period].loc[time_stamp:time_stamp + timedelta(days=150)]
            bb_signals_period = bollinger_band_data[period].loc[time_stamp:time_stamp + timedelta(days=150)]

            # Iterate over each row in the Bollinger Band signals (day by day)
            for row_idx, row in bb_signals_period.iterrows():
                daily_balance_change = 0
                daily_portfolio_value = 0
                
                for col_idx, signal in enumerate(row):
                    ticker = tickers[col_idx]  # Correctly get ticker for each column
                    adj_close_price = adj_close_period.loc[row_idx, ticker]  # Get corresponding adjusted close price

                    # initialize the tracking of stocks for current ticker
                    stock_tracker = all_data['Stock Tracker'][period][ticker]

                    # Handle Buy action
                    if signal == 'Buy':
                        amount_to_buy = percent_to_buy * account_balance
                        if account_balance >= amount_to_buy:
                            # Calculate shares to buy
                            shares_to_buy = amount_to_buy / adj_close_price
                            shares_number[ticker] += shares_to_buy
                            shares_value[ticker] = shares_number[ticker] * adj_close_price

                            # Track investment for the current period
                            stock_tracker = stock_tracker.append({
                                'Date': row_idx,
                                'Share Price': adj_close_price,
                                'Signal': 'Buy',
                                'Buy/Sell Amount ($)': amount_to_buy,
                                'Buy/Sell Number of Shares': shares_to_buy,
                                'Shares ($) Ownership': shares_value[ticker],
                                'Shares Ownership': shares_number[ticker]
                            }, ignore_index=True)

                            # Update account balance after buying
                            account_balance -= amount_to_buy

                    # Handle Sell action
                    elif signal == 'Sell':
                        amount_to_sell = percent_to_sell * shares_value[ticker]
                        if shares_value[ticker] >= amount_to_sell:
                            # Calculate shares to sell
                            shares_to_sell = amount_to_sell / adj_close_price
                            shares_number[ticker] -= shares_to_sell
                            shares_value[ticker] = shares_number[ticker] * adj_close_price

                            # Track the sell action
                            stock_tracker = stock_tracker.append({
                                'Date': row_idx,
                                'Share Price': adj_close_price,
                                'Signal': 'Sell',
                                'Buy/Sell Amount ($)': amount_to_sell,
                                'Buy/Sell Number of Shares': shares_to_sell,
                                'Shares ($) Ownership': shares_value[ticker],
                                'Shares Ownership': shares_number[ticker]
                            }, ignore_index=True)

                            # Update account balance after selling
                            account_balance += amount_to_sell

                    # Handle Hold action (no action taken)
                    else:
                        # Track the hold state
                        stock_tracker = stock_tracker.append({
                            'Date': row_idx,
                            'Share Price': adj_close_price,
                            'Signal': 'Hold',
                            'Buy/Sell Amount ($)': 0,
                            'Buy/Sell Number of Shares': 0,
                            'Shares ($) Ownership': shares_value[ticker],
                            'Shares Ownership': shares_number[ticker]
                        }, ignore_index=True)

                    # Save the updated tracker back to portfolio summary
                    all_data['Stock Tracker'][period][ticker] = stock_tracker.copy()

                # Calculate total portfolio value for all tickers for the day
                portfolio_value = sum(shares_value.values())

                # Total value (account balance + portfolio value)
                total_value = account_balance + portfolio_value

                # Calculate profit (difference from initial investment)
                profit = total_value - initial_investment

                # Track portfolio changes for the current day
                portfolio_tracker[period] = portfolio_tracker[period].append({
                    'Date': row_idx,
                    'Account Balance': account_balance,
                    'Portfolio Value': portfolio_value,
                    'Total Value': total_value,
                    'Profit': profit
                }, ignore_index=True)

            # Update the portfolio tracker for the period
            all_data['Portfolio Tracker'][period] = portfolio_tracker[period]

    # Return the complete portfolio summary for all periods and tickers
    return all_data


a = bb_band_roi(bb_bands_signals,adj_close_sector_etf,economic_cycle_periods,economic_cycle_periods_list,sector_etf_tickers,1,1000,0.05,0.05)

In [87]:
a['Stock Tracker']['expansion']['XLF'].head(50)

,Date,Share Price,Signal,Buy/Sell Amount ($),Buy/Sell Number of Shares,Shares ($) Ownership,Shares Ownership
0,2013-11-25,14.185604,Hold,0.000000,0.000000,0.000000,0.000000
1,2013-11-26,14.172426,Hold,0.000000,0.000000,0.000000,0.000000
2,2013-11-27,14.211968,Hold,0.000000,0.000000,0.000000,0.000000
3,2013-11-29,14.159234,Hold,0.000000,0.000000,0.000000,0.000000
4,2013-12-02,14.152646,Hold,0.000000,0.000000,0.000000,0.000000
5,2013-12-03,14.007628,Hold,0.000000,0.000000,0.000000,0.000000
6,2013-12-04,14.040585,Hold,0.000000,0.000000,0.000000,0.000000
7,2013-12-05,13.908745,Hold,0.000000,0.000000,0.000000,0.000000
8,2013-12-06,14.099915,Hold,0.000000,0.000000,0.000000,0.000000
9,2013-12-09,14.159234,Hold,0.000000,0.000000,0.000000,0.000000


In [89]:
a['Portfolio Tracker']['expansion'].head(50)

,Date,Account Balance,Portfolio Value,Total Value,Profit
0,2013-11-25,1000.000000,0.000000,1000.000000,0.000000
1,2013-11-26,950.000000,50.000000,1000.000000,0.000000
2,2013-11-27,902.500000,97.395114,999.895114,-0.104886
3,2013-11-29,902.500000,97.395114,999.895114,-0.104886
4,2013-12-02,902.500000,97.395114,999.895114,-0.104886
5,2013-12-03,902.500000,97.395114,999.895114,-0.104886
6,2013-12-04,902.500000,97.395114,999.895114,-0.104886
7,2013-12-05,857.375000,142.520114,999.895114,-0.104886
8,2013-12-06,857.375000,142.520114,999.895114,-0.104886
9,2013-12-09,857.375000,142.520114,999.895114,-0.104886
